In [1]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install scikit-learn

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Step 1: Load the dataset
import pandas as pd
from IPython.display import display

# Load the dataset (ensure the path is correct for your environment)
df = pd.read_csv("nfl_stadiums.csv", encoding="ISO-8859-1")

# Display the first few rows of the dataframe to understand its structure
print("Initial data preview:")
display(df[0:100])

Initial data preview:


,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_zipcode,stadium_weather_type,stadium_capacity,stadium_surface,stadium_weather_station,stadium_weather_station_name,stadium_latitude,stadium_longitude,stadium_azimuthangle,stadium_elevation
0,Acrisure Stadium,"Pittsburgh, PA",2001.0,NaN,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,USW00094823,"PITTSBURGH ASOS, PA US",40.484600,-80.214400,NaN,366.7
1,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,indoor,72000,FieldTurf,NaN,NaN,29.416944,-98.478889,0.0,NaN
2,Allegiant Stadium,"Paradise, NV",2020.0,NaN,indoor,NaN,NaN,indoor,65000,Grass,NaN,NaN,36.090750,-115.183722,0.0,NaN
3,Allianz Arena,"Munich, Germany",NaN,NaN,outdoor,NaN,NaN,moderate,"75,024",Grass,NaN,NaN,NaN,NaN,NaN,NaN
4,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,NaN,NaN,warm,NaN,NaN,NaN,NaN,30.323889,-81.637500,16.7,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Stanford Stadium,"Palo Alto, CA",NaN,NaN,NaN,NaN,NaN,moderate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,State Farm Stadium,"Glendale, AZ",2006.0,NaN,retractable,"1 Cardinals Dr, Glendale, AZ 85305",85305,indoor,"63,400",Grass,US1AZMR0451,"TEMPE 3.6 NNW, AZ US",33.528000,-112.263000,330.0,375.2
97,StubHub Center,"Los Angeles, CA",2017.0,NaN,outdoor,"18400 S Avalon Blvd, Carson, CA 90746",90746,warm,27000,Grass,USW00023174,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",33.938000,-118.388800,NaN,29.6
98,Sun Devil Stadium,"Tempe, AZ",1988.0,2005.0,outdoor,"500 E Veterans Way, Tempe, AZ 85287",85287,warm,NaN,NaN,US1AZMR0451,"TEMPE 3.6 NNW, AZ US",33.426389,-111.932500,14.2,375.2


In [3]:
# Step 2: Identify and handle missing data

# Check for missing values in the dataset
print("\nMissing values in each column:")
print(df.isnull().sum())

# Fill missing values for numerical columns with the median to reduce the impact of outliers
numerical_columns = ["stadium_open", "stadium_close", "stadium_capacity", 'stadium_elevation']
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")  # Ensure columns are numeric
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)
    print(f"Filled missing values in {col} with median: {median_value}")

# Fill missing values for categorical columns with a placeholder 'Missing'
categorical_columns = df.select_dtypes(include=["object"]).columns
df[categorical_columns] = df[categorical_columns].fillna("Missing")

# Verify that there are no missing values left
print("\nMissing values after handling:")
print(df.isnull().sum())


Missing values in each column:
stadium_name                        0
stadium_location                    1
stadium_open                       30
stadium_close                      79
stadium_type                       11
stadium_address                    18
stadium_weather_station_zipcode    19
stadium_weather_type                3
stadium_capacity                   66
stadium_surface                    52
stadium_weather_station            58
stadium_weather_station_name       57
stadium_latitude                   35
stadium_longitude                  35
stadium_azimuthangle               61
stadium_elevation                  55
dtype: int64
Filled missing values in stadium_open with median: 1993.5
Filled missing values in stadium_close with median: 2000.0
Filled missing values in stadium_capacity with median: 67620.0
Filled missing values in stadium_elevation with median: 163.4

Missing values after handling:
stadium_name                        0
stadium_location                   

/var/folders/z6/fz7p_t5j2zdcx0r2h3wlzzs80000gn/T/ipykernel_35326/956778976.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_value, inplace=True)
/var/folders/z6/fz7p_t5j2zdcx0r2h3wlzzs80000gn/T/ipykernel_35326/956778976.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [4]:
# Step 3: Identify and handle outliers

# Calculate the IQR for 'stadium_capacity' to identify outliers
Q1 = df["stadium_capacity"].quantile(0.25)
Q3 = df["stadium_capacity"].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
df = df[
    (df["stadium_capacity"] >= lower_bound) & (df["stadium_capacity"] <= upper_bound)
]

# Reset the index after removing outliers
df.reset_index(drop=True, inplace=True)

print("\nData after removing outliers:")
print(df.describe())


Data after removing outliers:
       stadium_open  stadium_close  stadium_capacity  stadium_latitude  \
count    114.000000     114.000000             114.0         79.000000   
mean    1985.635965    1999.868421           67620.0         37.779202   
std       19.977674       7.717940               0.0          6.804528   
min     1909.000000    1970.000000           67620.0         19.302911   
25%     1973.000000    2000.000000           67620.0         33.133194   
50%     1993.500000    2000.000000           67620.0         39.100000   
75%     1999.000000    2000.000000           67620.0         41.976650   
max     2016.000000    2016.000000           67620.0         51.604444   

       stadium_longitude  stadium_azimuthangle  stadium_elevation  
count          79.000000             54.000000         114.000000  
mean          -85.481706            178.359259         183.148246  
std            24.171554            149.639682         252.539694  
min          -122.409500      

In [5]:
# Step 4: Correct any data errors

# Replace 'Unknown' with NaN in 'stadium_azimuthangle' and fill with the median
df["stadium_azimuthangle"] = pd.to_numeric(df["stadium_azimuthangle"], errors="coerce")
median_azimuthangle = df["stadium_azimuthangle"].median()
df["stadium_azimuthangle"].fillna(median_azimuthangle, inplace=True)
print(
    f"\nCorrected 'stadium_azimuthangle' by filling missing values with median: {median_azimuthangle}"
)

# Display the first few rows after cleaning
print("\nData after cleaning:")
print(df.head())


Corrected 'stadium_azimuthangle' by filling missing values with median: 189.75

Data after cleaning:
       stadium_name   stadium_location  stadium_open  stadium_close  \
0  Acrisure Stadium     Pittsburgh, PA        2001.0         2000.0   
1     Allianz Arena    Munich, Germany        1993.5         2000.0   
2    Alltel Stadium   Jacksonville, FL        1993.5         2000.0   
3    Alumni Stadium  Chestnut Hill, MA        1993.5         2000.0   
4   Anaheim Stadium        Anaheim, CA        1980.0         1994.0   

  stadium_type                           stadium_address  \
0      outdoor  100 Art Rooney Ave, Pittsburgh, PA 15212   
1      outdoor                                   Missing   
2      Missing                                   Missing   
3      outdoor     Perimeter Rd, Chestnut Hill, MA 02467   
4      outdoor  2000 E Gene Autry Way, Anaheim, CA 92806   

  stadium_weather_station_zipcode stadium_weather_type  stadium_capacity  \
0                           15212 

/var/folders/z6/fz7p_t5j2zdcx0r2h3wlzzs80000gn/T/ipykernel_35326/1992291883.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["stadium_azimuthangle"].fillna(median_azimuthangle, inplace=True)


In [6]:
# Feature Engineering

# Step 5: Create a new feature 'stadium_age'
# get current year
current_year = pd.Timestamp.now().year

# Fill missing values in 'stadium_close' with the current year
if df["stadium_close"].isnull().sum() > 0:
    df["stadium_close"].fillna(current_year, inplace=True)

# Create the new feature 'stadium_age' by subtracting 'stadium_open' from 'stadium_close'
df["stadium_age"] = df["stadium_close"] - df["stadium_open"]

# Replace negative values with 0
df["stadium_age"] = df["stadium_age"].apply(lambda x: x if x >= 0 else 0)
print("\nBiến mới 'stadium_age':")
print(df["stadium_age"].head())


Biến mới 'stadium_age':
0     0.0
1     6.5
2     6.5
3     6.5
4    14.0
Name: stadium_age, dtype: float64


In [7]:
# Encoding Categorical Variables

# Step 6: Perform one-hot encoding on categorical variables
df = pd.get_dummies(
    df,
    columns=["stadium_type", "stadium_weather_type", "stadium_surface"],
    prefix=["type", "weather", "surface"],
)

print("\nData after one-hot encoding:")
print(df.head())


Data after one-hot encoding:
       stadium_name   stadium_location  stadium_open  stadium_close  \
0  Acrisure Stadium     Pittsburgh, PA        2001.0         2000.0   
1     Allianz Arena    Munich, Germany        1993.5         2000.0   
2    Alltel Stadium   Jacksonville, FL        1993.5         2000.0   
3    Alumni Stadium  Chestnut Hill, MA        1993.5         2000.0   
4   Anaheim Stadium        Anaheim, CA        1980.0         1994.0   

                            stadium_address stadium_weather_station_zipcode  \
0  100 Art Rooney Ave, Pittsburgh, PA 15212                           15212   
1                                   Missing                         Missing   
2                                   Missing                         Missing   
3     Perimeter Rd, Chestnut Hill, MA 02467                            2467   
4  2000 E Gene Autry Way, Anaheim, CA 92806                           92806   

   stadium_capacity stadium_weather_station stadium_weather_station_

In [8]:
# Data Normalization

# Step 7: Apply Min-Max scaling to numerical columns
numerical_cols = [
    "stadium_capacity",
    "stadium_open",
    "stadium_close",
    "stadium_azimuthangle",
    "stadium_elevation",
    "stadium_age",
]

# Initialize the MinMaxScaler
minmax_scaler = MinMaxScaler()

# Apply Min-Max scaling to the numerical columns
df[numerical_cols] = minmax_scaler.fit_transform(df[numerical_cols])

print("\nSau khi chuẩn hóa dữ liệu:")
print(df[numerical_cols].head())


Sau khi chuẩn hóa dữ liệu:
   stadium_capacity  stadium_open  stadium_close  stadium_azimuthangle  \
0               0.0      0.859813       0.652174              0.529732   
1               0.0      0.789720       0.652174              0.529732   
2               0.0      0.789720       0.652174              0.046622   
3               0.0      0.789720       0.652174              0.529732   
4               0.0      0.663551       0.521739              0.529732   

   stadium_elevation  stadium_age  
0           0.226730     0.000000  
1           0.100410     0.071429  
2           0.002672     0.071429  
3           0.100410     0.071429  
4           0.100410     0.153846  


In [9]:
# Data Visualization
display(df)

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_address,stadium_weather_station_zipcode,stadium_capacity,stadium_weather_station,stadium_weather_station_name,stadium_latitude,...,weather_cold,weather_indoor,weather_moderate,weather_warm,surface_FieldTurf,surface_Grass,"surface_Grass, Turf (1969-1970)","surface_Grass, Turf (1970-1971)","surface_Grass, Turf (1971-1974)",surface_Missing
0,Acrisure Stadium,"Pittsburgh, PA",0.859813,0.652174,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,0.0,USW00094823,"PITTSBURGH ASOS, PA US",40.484600,...,True,False,False,False,False,True,False,False,False,False
1,Allianz Arena,"Munich, Germany",0.789720,0.652174,Missing,Missing,0.0,Missing,Missing,NaN,...,False,False,True,False,False,True,False,False,False,False
2,Alltel Stadium,"Jacksonville, FL",0.789720,0.652174,Missing,Missing,0.0,Missing,Missing,30.323889,...,False,False,False,True,False,False,False,False,False,True
3,Alumni Stadium,"Chestnut Hill, MA",0.789720,0.652174,"Perimeter Rd, Chestnut Hill, MA 02467",2467,0.0,Missing,Missing,NaN,...,True,False,False,False,False,True,False,False,False,False
4,Anaheim Stadium,"Anaheim, CA",0.663551,0.521739,"2000 E Gene Autry Way, Anaheim, CA 92806",92806,0.0,Missing,Missing,NaN,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,War Memorial Stadium,"Buffalo, NY",0.476636,0.043478,"285 Dodge Street, Buffalo, NY 14208",14208,0.0,Missing,Missing,NaN,...,True,False,False,False,False,False,False,False,False,True
110,Wembley Stadium,"London, UK",0.915888,0.652174,"Wembley, London HA9 0WS, United Kingdom","Heathrow, UK",0.0,Missing,Missing,51.555833,...,False,False,True,False,False,False,False,False,False,True
111,Wrigley Field,"Chicago, IL",0.102804,0.000000,"1060 West Addison Street, Chicago, IL 60613",60613,0.0,Missing,Missing,NaN,...,True,False,False,False,False,False,False,False,False,True
112,Yale Bowl,"New Haven, CT",0.789720,0.652174,"276 Derby Ave, West Haven, CT 06516",6516,0.0,Missing,Missing,NaN,...,True,False,False,False,False,True,False,False,False,False
